In [32]:
import numpy as np
import utilities as ut
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [33]:
# Change dataset name if needed
dataset = 'UJI'
pred_w = 'block' ## 'block' or 'floor','building'
data = np.load(f'data/dataset_{dataset}.npz', allow_pickle=True)

In [34]:
# X_train = data['X_train']
X_train = ut.normalize_zscore_rowwise(data['X_train'])
Y_train = data['Y_train']
# X_test = data['X_test']
X_test = ut.normalize_zscore_rowwise(data['X_test'])
Y_test = data['Y_test']
block_info = data['block_info'][0]

In [35]:
if pred_w == 'block':
    pred_col = -1
elif pred_w == 'floor':
    pred_col = 0
elif pred_w == 'building':
    pred_col = 1

# Extract target: last column (e.g., block ID, class label)
y_train = Y_train[:, pred_col].astype(int)
y_test = Y_test[:, pred_col].astype(int)

# Encode labels if they are not already 0-based integers
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_enc, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_enc, dtype=torch.long)

# Create Dataloaders
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)

In [36]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.net(x)

In [37]:
# Model setup
input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train_enc))
model = MLP(input_dim, output_dim).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
epochs = 30
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")


Epoch 1 - Loss: 941.0826
Epoch 2 - Loss: 502.1230
Epoch 3 - Loss: 391.1057
Epoch 4 - Loss: 330.5397
Epoch 5 - Loss: 289.6150
Epoch 6 - Loss: 257.8154
Epoch 7 - Loss: 234.0756
Epoch 8 - Loss: 217.8901
Epoch 9 - Loss: 197.9371
Epoch 10 - Loss: 183.6172
Epoch 11 - Loss: 176.4790
Epoch 12 - Loss: 162.7554
Epoch 13 - Loss: 155.9389
Epoch 14 - Loss: 146.8833
Epoch 15 - Loss: 136.2548
Epoch 16 - Loss: 133.0690
Epoch 17 - Loss: 124.3031
Epoch 18 - Loss: 121.7543
Epoch 19 - Loss: 117.1591
Epoch 20 - Loss: 111.8874
Epoch 21 - Loss: 106.0833
Epoch 22 - Loss: 105.9190
Epoch 23 - Loss: 106.7971
Epoch 24 - Loss: 98.0182
Epoch 25 - Loss: 96.5195
Epoch 26 - Loss: 95.8463
Epoch 27 - Loss: 90.1593
Epoch 28 - Loss: 90.1315
Epoch 29 - Loss: 85.6018
Epoch 30 - Loss: 84.4764


In [38]:
model.eval()
correct = total = 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        preds = model(xb).argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)


In [39]:
model.eval()
X_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
with torch.no_grad():
    logits = model(X_tensor)
    preds = logits.argmax(dim=1).cpu().numpy()

# Decode predicted block IDs
pred_block_ids = label_encoder.inverse_transform(preds)
true_block_ids = Y_test[:, pred_col].astype(int)

# Compute predicted and true coordinates
pred_coords = np.array([[block_info[b]['x'], block_info[b]['y']] for b in pred_block_ids])
true_coords = Y_test[:, -4:-2]  # assuming these are [x, y] columns

# Localization errors
errors = np.linalg.norm(pred_coords - true_coords, axis=1)

# Metrics
acc = np.mean(pred_block_ids == true_block_ids)
mle = np.mean(errors)

print(f" MLE: {mle:.2f} meters")
print(f" 75th percentile error: {np.percentile(errors, 75):.2f}m")
print(f" 90th percentile error: {np.percentile(errors, 90):.2f}m")

 MLE: 15.66 meters
 75th percentile error: 13.45m
 90th percentile error: 24.98m
